<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/inferenceTotalSegmentatorNotebookTroubleshootingMLflag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**This Notebook does the second step in the split workflow on Terra**

The NIFTI files generated from first step are provided as input for Inference.


Please cite:

Jakob Wasserthal, Manfred Meyer, Hanns-Christian Breit, Joshy Cyriac, Shan Yang, & Martin Segeroth. (2022). TotalSegmentator: robust segmentation of 104 anatomical structures in CT images. https://doi.org/10.48550/arXiv.2208.05868

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nat Methods 18, 203–211 (2021). https://doi.org/10.1038/s41592-020-01008-z

###**Installing Packages**

In [4]:
%%capture
#Installing lz4 for compression/decompression
#xvfb required for previewing Totalsegmentator png images
!apt-get install lz4 xvfb

In [5]:
%%capture
#Installing Totalsegmentator for Inference
#pyradiomics required for extracting radiomics features 
#nvidia-ml-py3 for capturing gpu utilization
!pip install TotalSegmentator pyradiomics nvidia-ml-py3

###**Importing Packages**

In [39]:
import glob
import os
import sys
import shutil
from pathlib import Path
import time
import subprocess
from concurrent.futures import ThreadPoolExecutor
from time import sleep
from datetime import datetime
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import nvidia_smi


curr_dir   = Path().absolute()

print(time.asctime(time.localtime()))
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Wed Apr 26 16:34:25 2023

Current directory :/content
Python version    : 3.9.16 (main, Dec  7 2022, 01:11:51) 


###**Parameters for Papermill**

In [40]:
converterType='dcm2niix'
niftiFilePath=''

###**Local testing**

In [41]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!gdown "https://drive.google.com/uc?id=1GdtvCeJk2qTARWOMCeNLz97ltF4plkDq" 
niftiFilePath= glob.glob("*.lz4")[0]


Downloading...
From: https://drive.google.com/uc?id=1GdtvCeJk2qTARWOMCeNLz97ltF4plkDq
To: /content/niftiFilesForTroubleshooting.tar.lz4
100% 36.5M/36.5M [00:00<00:00, 206MB/s]


###**Decompressing NIFTI files from first step**

In [43]:
!lz4 -d --rm {niftiFilePath} -c | tar  --strip-components=0  -xvf -

dcm2niix/1.2.840.113654.2.55.100875189782210690344207306235124901243/
dcm2niix/1.2.840.113654.2.55.100875189782210690344207306235124901243/1.2.840.113654.2.55.100875189782210690344207306235124901243_0_OPA_GE_LSQX_STANDARD_360_2.5_120_na_na_na_19990102000000_2.nii.gz


###**TotalSegmentator Troubleshooting --ml flag**

In [58]:
try:
  shutil.rmtree('metadataWithMLflag')
  shutil.rmtree('metadataWithOUTMLflag')
  shutil.rmtree('segmentations')
  shutil.rmtree('segmentationsWithMLflag')
except OSError:
  pass

os.mkdir('metadataWithMLflag')
os.mkdir('metadataWithOUTMLflag')
os.mkdir('segmentationsWithMLflag')


####**With ML Flag**

In [59]:
#this command does not create 'segmentations' folder and 
#does not compress the output nifti file
#create an empty file for radiomics features

!TotalSegmentator -i /content/dcm2niix/1.2.840.113654.2.55.100875189782210690344207306235124901243/1.2.840.113654.2.55.100875189782210690344207306235124901243_0_OPA_GE_LSQX_STANDARD_360_2.5_120_na_na_na_19990102000000_2.nii.gz \
  -o segmentations --ml --fast --preview --radiomics

!mv segmentations.nii segmentationsWithMLflag/
!mv preview_total.png  statistics_radiomics.json metadataWithMLflag/


If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Using 'fast' option: resampling to lower resolution (3mm)
Resampling...
  Resampled in 2.36s
Predicting...
  Predicted in 8.15s
Generating preview...
  Generated in 5.41s
Resampling...
Saving segmentations...
  Saved in 0.44s
Calculating radiomics...
0it [00:00, ?it/s]
  calculated in 0.10s


####**WithOUT ML Flag**

In [60]:
#this command CREATES 'segmentations' folder and 
#compresses each body part's  output nifti file
#calculates radiomics features correctly

!TotalSegmentator -i /content/dcm2niix/1.2.840.113654.2.55.100875189782210690344207306235124901243/1.2.840.113654.2.55.100875189782210690344207306235124901243_0_OPA_GE_LSQX_STANDARD_360_2.5_120_na_na_na_19990102000000_2.nii.gz \
  -o segmentationsWithMLflag --fast --preview --radiomics


!mv segmentationsWithMLflag/preview_total.png segmentationsWithMLflag/statistics_radiomics.json metadataWithOUTMLflag/


If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Using 'fast' option: resampling to lower resolution (3mm)
Resampling...
  Resampled in 2.29s
Predicting...
  Predicted in 9.01s
Generating preview...
  Generated in 5.38s
Resampling...
Saving segmentations...
100% 104/104 [00:32<00:00,  3.18it/s]
  Saved in 33.82s
Calculating radiomics...
  5% 5/104 [00:52<17:17, 10.48s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 11% 11/104 [01:45<15:12,  9.81s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 12% 12/104 [01:47<11:25,  7.45s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 12% 13/104 [01:49<08:50,  5.83s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 13% 14/104 [01:51<07:01,  4.69s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 14% 15/104 [01:53<05:57,  4.01s/it]WARNING: Entire mask is 0 or 1. Setting all features to 0
 15% 16/104 [01:55<05:04,  3.46s/it]WARNING: Entire mask is 0 or 1. S